In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

from pylab import rcParams
rcParams['figure.figsize'] = 12,8

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns


import pandas as pd
import numpy as np

from datetime import datetime, timedelta

import re

import os
from glob import glob
from tqdm import tqdm

import yaml
from yaml import dump
import uuid
import itertools
from shutil import copy2

import matplotlib.pyplot as plt

In [3]:
import xgboost
from xgboost import XGBRegressor

from sklearn.ensemble import RandomForestRegressor

from lightgbm import LGBMRegressor

from prophet import Prophet

In [4]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [5]:
import warnings
warnings.simplefilter(action="ignore")

## Forecasting

#### initialize all required valiables, prepare datasets

In [6]:
def init():

    train_end = datetime(2025, 8, 31, 23)
    test_start = datetime(2025, 9, 1, 0)
    test_end = datetime(2025, 9, 14, 23)
    
    train_features_set = [
        ['month', 'year_day', 'is_day', 'sunshine_duration', 'temperature_min_3_weeks', 'temperature_max_3_weeks', 'temperature_mean_3_weeks',
         'temperature_min_3_years', 'temperature_max_3_years', 'temperature_mean_3_years', 'cloud_cover_mean_7_days', 'pressure_msl_mean_7_days'],
        ['month', 'year_day', 'is_day', 'sunshine_duration', 'temperature_min_3_weeks', 'temperature_max_3_weeks', 'temperature_mean_3_weeks',
        'relative_humidity_min_7_days', 'temperature_min_7_days', 'temperature_mean_7_days', 'temperature_lag_168'],
        ['month', 'year_day', 'is_day', 'sunshine_duration', 'temperature_min_3_weeks', 'temperature_max_3_weeks', 'temperature_mean_3_weeks']
    ]

    
    date_parse = lambda dates: pd.to_datetime(dates)
    path = f"/masters_diploma/processed_data/history_weather.csv"
    
    full_set = pd.read_csv(
        path,
        parse_dates=["date"],
        date_parser=date_parse,
        index_col=["date"],
    )
    

    full_set = full_set[:test_end].fillna(0)
#     full_set = future_target(full_set, test_start, test_end)
    
    
#     test_start = datetime(full_set.loc[test_start:].index[0].year, full_set.loc[test_start:].index[0].month, full_set.loc[test_start:].index[0].day)
#     train_end = datetime(full_set.loc[:train_end].index[-1].year, full_set.loc[:train_end].index[-1].month, full_set.loc[:train_end].index[-1].day)
    
    
    return full_set, train_end, test_start, test_end, train_features_set

In [7]:
def future_target(df, date_start, date_end):
    
    date_range = pd.date_range(date_start, date_end, freq='H')
    
    add_df = pd.DataFrame(index=date_range, columns=df.columns)
    
#     add_df
    
    df = pd.concat([df, add_df])
    
    return df    

In [8]:
def models_hyperparameter_random_forest():

    depth_list = [4, 5, 6, 7]
    n_estimators_list = [50, 100, 200, 500, 1000]
    
    hyperparameters_for_model = []
    
    for depth, n_estimators in itertools.product(depth_list, n_estimators_list):
        hyperparameters_for_model.append({
                        'n_estimators': n_estimators,
                        'n_jobs': -1,
                        'random_state': 2,
                        'max_depth': depth,
            })

    return hyperparameters_for_model


def models_hyperparameter_xgboost():

    depth_list = [5, 7, 9]
    n_estimators_list = [50, 100, 200, 500, 1000]
    
    hyperparameters_for_model = []
    
    for depth, n_estimators in itertools.product(depth_list, n_estimators_list):
        hyperparameters_for_model.append({
                    'n_estimators': n_estimators,
                    'n_jobs': -1,
                    'max_depth': depth,
                    'eta': 0.3,
                    'booster': 'gbtree',
                    'objective': 'reg:squarederror',
                    'eval_metric': 'rmse',
                    'subsample': 1,
                    'colsample_bytree': 1,
                    'min_child_weight': 1,
                    'random_state': 2,
            })

    return hyperparameters_for_model


def models_hyperparameter_lgbm():

    
    depth_list = [6, 7]
    n_estimators_list = [10, 50, 100]
    
    hyperparameters_for_model = []
    
    for depth, n_estimators in itertools.product(depth_list, n_estimators_list):
        hyperparameters_for_model.append({
                    'n_estimators': n_estimators,
                    'n_jobs': -1,
                    'max_depth': depth,
                    'eta': 0.3,
                    'random_state': 2,
                    'objective': 'binary',
                    'verbosity': -1,
                    'metric': 'binary', 
            })

    return hyperparameters_for_model


def models_hyperparameter_prophet():
    
    hyperparameters_for_model.append({})

    seasonality_list = ["seasonality_yearly", "seasonality_daily"]
    season_list = ["additive", "multiplicative"]
    
    hyperparameters_for_model = []
    
    for seasonality, season in itertools.product(seasonality_list, season_list):
        hyperparameters_for_model.append({
                        'growth': "logistic",
                        'season': season,
                        f'{seasonality}': True,
            })

    return hyperparameters_for_model

In [9]:
def define_parameters(train_end, test_start, test_end, train_features_set, forecast_steps, models_dict):
    
    list_of_configs = []
    
    model = None
    
#     for duration in [90, 240]:
    for train_start in [datetime(2020, 1, 1), datetime(2024, 1, 1), datetime(2023, 9, 1)]:
        for md in models_dict.values():
            if md == 'random_forest':
                hyperparameters_for_model = models_hyperparameter_random_forest()
            elif md == 'xgboost':
                hyperparameters_for_model = models_hyperparameter_xgboost()
            elif md == 'lightgbm':
                hyperparameters_for_model = models_hyperparameter_lgbm()
            elif md == 'prophet':
                hyperparameters_for_model = models_hyperparameter_prophet()
            else:
                print('Unknown model')
                return


            for hp in hyperparameters_for_model:

                if md == 'random_forest':
                    model = RandomForestRegressor(**hp)
                elif md == 'xgboost':
                    model = XGBRegressor(**hp)
                elif md == 'lightgbm':
                    model = LGBMRegressor(**hp)
                elif md == 'prophet':
                    model = Prophet()
    #                 model = Prophet(**hp)
                else:
                    print('Unknown model')

                for train_features in train_features_set:
                    config = {
                        'unique_uuid': str(uuid.uuid1()),
                        'train_start': train_start,
                        'train_end': train_end,
                        'test_start': test_start,
                        'test_end': test_end,
    #                     'duration_training_history': duration,
                        'target_column': 'temperature',
                        'train_features': train_features,
                        'path_to_result': f'/masters_diploma/',
                        'forecast_days': forecast_steps,
        #                 'hour_mean_value': 5,
                        'model_name': md,
                        'model': model,
                        'model_hyperparameters': hp,
                    }

                    list_of_configs.append(config.copy())
    
    return list_of_configs

#### functions used in wfv service

In [10]:
def data(day, X_full_set, y_full_set, train_start, config, forecast_steps):

    X_train = X_full_set.loc[train_start:config["train_end"]]
    X_test = X_full_set.loc[config["test_start"]+timedelta(days=day): config["test_start"]+timedelta(days=day+forecast_steps, hours=23)]
    
    y_train = y_full_set.loc[train_start:config["train_end"]]
    y_test = y_full_set.loc[config["test_start"]+timedelta(days=day): config["test_start"]+timedelta(days=day+forecast_steps, hours=23)]
    
    return X_train, X_test, y_train, y_test

In [11]:
def standardize_mean_values(day, df_test, df_train, full_set, config):
    
    agg_cols = [col for col in config['train_features'] if (col.startswith(f"{config['target_column']}_m")) & (col.endswith('days'))]

    if agg_cols:
        for agg in agg_cols:
            if agg in df_test.columns:
                try:
#                 print(config["test_start"]+timedelta(days=day), df_test.loc[config["test_start"]+timedelta(days=day), agg])
                    num = df_test.loc[config["test_start"]+timedelta(days=day), agg]

                except KeyError as e:
                    num = df_train[agg].iloc[-1]

                finally:

                    _df = df_test.loc[config["test_start"]+timedelta(days=day):, agg]
                    _df = _df.replace(_df.values, num)

#                 print(df_test.loc[config["test_start"]+timedelta(days=day):, agg], _df.values.ravel())

                    df_test.loc[config["test_start"]:, agg] = _df.values.ravel()
    else:
        pass

    
    return df_test

In [12]:
def estimations(day, df_stats, y_pred_df, y_test, config):
    
    dates = y_test.index
    
    for date in dates:
        step_day = int((date-(config["test_start"]+timedelta(days=day))).days)

        try:
            pred = y_pred_df.loc[date, 0]
            real = y_test.loc[date, config['target_column']]

            err = abs(pred / real - 1) * 100

            df_stats.loc[date, f'd-{step_day}' + '_total_abs_error'] = np.round(abs(pred-real))
            df_stats.loc[date, f'd-{step_day}' + '_total_relative_error'] = np.round(abs(pred / real - 1), 4) * 100
            df_stats.loc[date, f'd-{step_day}' + '_more_5'] = 1 if (err > 5) else 0
            df_stats.loc[date, f'd-{step_day}' + '_more_10'] = 1 if (err > 10) else 0
            
        except ZeroDivisionError as e:
            print(e)

            df_stats.loc[date, :] = 0

        except KeyError as e:
            print(e)

            df_stats.loc[date, :] = 0
    
    return df_stats

In [13]:
def write_predictions(day, forecast_steps, y_pred_df, config, research_task_uuid):
    
    for step in range(forecast_steps+1):
        try:
            pred = y_pred_df.iloc[24*step:24*(step+1), 0].dropna().sort_index()
            pred.index.name = 'date_time'

            path_to_files = os.path.join(config['path_to_result'], "forecast", config['model_name'], 
                                         f"research_task_{research_task_uuid}", 
                                         f"{config['model_name']}_{config['unique_uuid']}")
            if not os.path.isdir(path_to_files):
                os.makedirs(path_to_files)
                
            file_name = os.path.join(path_to_files, 
                    f"forecast_d-{step}_{config['model_name']}_{(config['test_start']+timedelta(days=day)).strftime('%Y-%m-%d')}_({pred.index[0].strftime('%Y-%m-%d')}).csv")

            pd.DataFrame(pred).to_csv(file_name)

        except KeyError:
            pass
        
        except IndexError:
            pass

### wfv service

#### *add train_features grid (day_means, date_features, lag_features etc.)

#### *add checking LightGBM, Random_Forest, Linear_Classifier etc. 

#### *add lag_features (? check whats better: those or "mean"s) 

#### *am I able to add some factors like inflation/company's profit etc.

In [14]:
def run_wfv(full_set: pd.DataFrame, config: dict, research_task_uuid: str, forecast_steps: int, models_dict: dict):
    
    X_full_set = full_set.loc[:, config['train_features']]
    y_full_set = full_set.loc[:, [config['target_column']]]
    
    if X_full_set.shape[0] != y_full_set.shape[0]:
        common_index = list(set(X_full_set.index) & set(y_full_set.index))
        common_index.sort()
        X_full_set = X_full_set.loc[common_index, :]
        y_full_set = y_full_set.loc[common_index, :]
    print(X_full_set.shape, y_full_set.shape)
    

    df_preds = pd.DataFrame()
    df_stats = pd.DataFrame()

    count_days = (test_end - test_start).days + 1
    
    
    model_name = config['model_name']
    print(model_name)
    
    model = config['model']

    unique_uuid = config['unique_uuid']
    
    if not os.path.isdir(config['path_to_result']):
        os.makedirs(config['path_to_result'])

    path_folder_result = os.path.join(config['path_to_result'], "wf_result", model_name,
                                      f"research_task_{research_task_uuid}")
    if not os.path.isdir(path_folder_result):
        os.makedirs(path_folder_result)
        
        

    for day in tqdm(range(count_days)):
        
        
        train_start = config.get('train_start', None)
        if train_start is None:
            if config.get('duration_training_history', None) is None:
                train_start = X_full_set.index[0]
                config['train_start'] = datetime(train_start.year, train_start.month, train_start.day)
            else:
                train_start = config['train_end'] + timedelta(days=i - config['duration_training_history'])

        try:

            X_train, X_test, y_train, y_test = data(day, X_full_set, y_full_set, train_start, config, forecast_steps)
            X_test = standardize_mean_values(day, X_test.copy(), X_train, full_set, config)

            y_pred = model.fit(X_train, y_train).predict(X_test)


            y_pred_df = pd.DataFrame(y_pred, index=y_test.index)
            
            write_predictions(day, forecast_steps, y_pred_df, config, research_task_uuid)

            df_stats = estimations(day, df_stats, y_pred_df, y_test, config)
#             print('\n\n')
        


        except KeyError as e:
            print(e)
            continue
            
        except ValueError as e:
            print(e)
            continue


    last_index = df_stats.index[-1]
    df_stats.loc[last_index, 'model_hyperparameters'] = str(config['model_hyperparameters'])
    df_stats.loc[last_index, 'train_features'] = str(config['train_features'])
    
    path_to_save_result_csv = os.path.join(path_folder_result, f'{model_name}_{unique_uuid}.csv')
    df_stats.round(2).to_csv(path_to_save_result_csv, date_format='%Y-%m-%d %H:%M:%S')
    
    config_to_save = config.copy()
    config_to_save.pop('model', None)
    with open(os.path.join(path_folder_result, f'{model_name}_{unique_uuid}.yaml'), 'w') as outfile:
        dump(config_to_save, outfile, default_flow_style=False)

In [15]:
models_list = ['XGBoost', 'LightGBM', 'Random_Forest']    #'Prophet'
models_dict = dict([("".join(re.findall('([A-Z])', k)).lower(), k.lower()) for k in models_list])

forecast_steps = 3        # means that forecast will be made on {n} futute days 
 

full_set, train_end, test_start, test_end, train_features_set = init()
print(f'train ends: {train_end}\t test starts: {test_start}')

_research_task_uuid = str(uuid.uuid1())
print(f'_research_task_uuid = {_research_task_uuid}\n')

configs = define_parameters(train_end, test_start, test_end, train_features_set, forecast_steps, models_dict)
print(f'count_configs {len(configs)} \n')

for i, _ in enumerate(configs):
    print(i, _['model_name'], '==', _, '\n')

    run_wfv(full_set, _, _research_task_uuid, forecast_steps, models_dict)

train ends: 2025-08-31 23:00:00	 test starts: 2025-09-01 00:00:00
_research_task_uuid = 4332760d-a140-11f0-a50a-c0e434d84b22

count_configs 369 

xgboost == {'unique_uuid': '4332760e-a140-11f0-9d95-c0e434d84b22', 'train_start': datetime.datetime(2020, 1, 1, 0, 0), 'train_end': datetime.datetime(2025, 8, 31, 23, 0), 'test_start': datetime.datetime(2025, 9, 1, 0, 0), 'test_end': datetime.datetime(2025, 9, 14, 23, 0), 'target_column': 'temperature', 'train_features': ['month', 'year_day', 'is_day', 'sunshine_duration', 'temperature_min_3_weeks', 'temperature_max_3_weeks', 'temperature_mean_3_weeks', 'temperature_min_3_years', 'temperature_max_3_years', 'temperature_mean_3_years', 'cloud_cover_mean_7_days', 'pressure_msl_mean_7_days'], 'path_to_result': '/masters_diploma/', 'forecast_days': 3, 'model_name': 'xgboost', 'model': XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             device=N

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:04<00:00,  2.95it/s]


xgboost == {'unique_uuid': '4332760f-a140-11f0-a343-c0e434d84b22', 'train_start': datetime.datetime(2020, 1, 1, 0, 0), 'train_end': datetime.datetime(2025, 8, 31, 23, 0), 'test_start': datetime.datetime(2025, 9, 1, 0, 0), 'test_end': datetime.datetime(2025, 9, 14, 23, 0), 'target_column': 'temperature', 'train_features': ['month', 'year_day', 'is_day', 'sunshine_duration', 'temperature_min_3_weeks', 'temperature_max_3_weeks', 'temperature_mean_3_weeks', 'relative_humidity_min_7_days', 'temperature_min_7_days', 'temperature_mean_7_days', 'temperature_lag_168'], 'path_to_result': '/masters_diploma/', 'forecast_days': 3, 'model_name': 'xgboost', 'model': XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             device=None, early_stopping_rounds=None, enable_categorical=False,
             eta=0.3, eval_metric='rmse', feature_types=None, gamma=None,
             grow_policy=None, importance_

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:03<00:00,  4.14it/s]


xgboost == {'unique_uuid': '43327610-a140-11f0-b28f-c0e434d84b22', 'train_start': datetime.datetime(2020, 1, 1, 0, 0), 'train_end': datetime.datetime(2025, 8, 31, 23, 0), 'test_start': datetime.datetime(2025, 9, 1, 0, 0), 'test_end': datetime.datetime(2025, 9, 14, 23, 0), 'target_column': 'temperature', 'train_features': ['month', 'year_day', 'is_day', 'sunshine_duration', 'temperature_min_3_weeks', 'temperature_max_3_weeks', 'temperature_mean_3_weeks'], 'path_to_result': '/masters_diploma/', 'forecast_days': 3, 'model_name': 'xgboost', 'model': XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             device=None, early_stopping_rounds=None, enable_categorical=False,
             eta=0.3, eval_metric='rmse', feature_types=None, gamma=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat

...

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [03:08<00:00, 13.43s/it]


random_forest == {'unique_uuid': '4332777d-a140-11f0-b0c0-c0e434d84b22', 'train_start': datetime.datetime(2023, 9, 1, 0, 0), 'train_end': datetime.datetime(2025, 8, 31, 23, 0), 'test_start': datetime.datetime(2025, 9, 1, 0, 0), 'test_end': datetime.datetime(2025, 9, 14, 23, 0), 'target_column': 'temperature', 'train_features': ['month', 'year_day', 'is_day', 'sunshine_duration', 'temperature_min_3_weeks', 'temperature_max_3_weeks', 'temperature_mean_3_weeks', 'relative_humidity_min_7_days', 'temperature_min_7_days', 'temperature_mean_7_days', 'temperature_lag_168'], 'path_to_result': '/masters_diploma/', 'forecast_days': 3, 'model_name': 'random_forest', 'model': RandomForestRegressor(max_depth=7, n_estimators=1000, n_jobs=-1, random_state=2), 'model_hyperparameters': {'n_estimators': 1000, 'n_jobs': -1, 'random_state': 2, 'max_depth': 7}} 

(93840, 11) (93840, 1)
random_forest


100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [02:30<00:00, 10.75s/it]


random_forest == {'unique_uuid': '4332777e-a140-11f0-8835-c0e434d84b22', 'train_start': datetime.datetime(2023, 9, 1, 0, 0), 'train_end': datetime.datetime(2025, 8, 31, 23, 0), 'test_start': datetime.datetime(2025, 9, 1, 0, 0), 'test_end': datetime.datetime(2025, 9, 14, 23, 0), 'target_column': 'temperature', 'train_features': ['month', 'year_day', 'is_day', 'sunshine_duration', 'temperature_min_3_weeks', 'temperature_max_3_weeks', 'temperature_mean_3_weeks'], 'path_to_result': '/masters_diploma/', 'forecast_days': 3, 'model_name': 'random_forest', 'model': RandomForestRegressor(max_depth=7, n_estimators=1000, n_jobs=-1, random_state=2), 'model_hyperparameters': {'n_estimators': 1000, 'n_jobs': -1, 'random_state': 2, 'max_depth': 7}} 

(93840, 7) (93840, 1)
random_forest


100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [01:26<00:00,  6.21s/it]


In [16]:
copy2('wfv.ipynb', f'/masters_diploma/archive/wfv_{str(uuid.uuid1())}_{datetime.now().strftime("%Y%m%d_%H%M%S")}')

'/masters_diploma/archive/wfv_ba4bfc8e-a163-11f0-8ad0-c0e434d84b22_20251004_235014'